In [4]:
# admit -> target -> 0또는 1 이므로 분류(classification) 문제

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/admission.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   admit   400 non-null    int64  
 1   gre     400 non-null    int64  
 2   gpa     400 non-null    float64
 3   rank    400 non-null    int64  
dtypes: float64(1), int64(3)
memory usage: 12.6 KB


In [7]:
df.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [8]:
y = df['admit'].values.reshape(-1,1)
X = df.drop(columns='admit',axis=1).values

In [9]:
X

array([[380.  ,   3.61,   3.  ],
       [660.  ,   3.67,   3.  ],
       [800.  ,   4.  ,   1.  ],
       ...,
       [460.  ,   2.63,   2.  ],
       [700.  ,   3.65,   2.  ],
       [600.  ,   3.89,   3.  ]])

In [10]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3)

In [11]:
# 정규화(0~1 값으로)
sc = MinMaxScaler()
# fit -> 모델링을 한다
X_train_sc = sc.fit_transform(X_train)
# transform -> 변형을 한다.
X_test_sc = sc.transform(X_test)

In [33]:
import tensorflow as tf
from tensorflow import keras
# 모델
from keras.models import Sequential
# 평탄화, 깊이
from keras.layers import Flatten, Dense, Dropout
# 옵티마이져
from keras.optimizers import SGD, Adam

from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Flatten, Dense, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.metrics import Precision, Recall, AUC

In [14]:
# seq_model = Sequential()
# seq_model.add(Flatten(input_shape=(3,)))  # 입력노드
# seq_model.add(Dense(units=10, activation='relu')) # 은닉층
# seq_model.add(Dense(units=10, activation='relu')) # 은닉층
# # 다중분류 (soft max) / 0 or 1 sigmoid
# seq_model.add(Dense(units=1, activation='sigmoid')) # 출력
# # loss option -> mean_squared_error / binary_crossentropy / categorical_crossentropy / mean_absolute_error
# seq_model.compile(optimizer=SGD(learning_rate=0.01),
#               loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
def lr_schedule(epoch):
    """Learning rate scheduler"""
    lr = 0.01
    if epoch > 50:
        lr = 0.001
    if epoch > 100:
        lr = 0.0001
    return lr


In [34]:
# Define the model
seq_model = Sequential([
    Flatten(input_shape=(3,)),
    Dense(units=32, activation='relu'),
    BatchNormalization(),
    Dense(units=16, activation='relu'),
    BatchNormalization(),
    Dense(units=1, activation='sigmoid')
])

# Compile the model
seq_model.compile(optimizer=Adam(learning_rate=0.01),
                  loss='binary_crossentropy',
                  metrics=['accuracy', Precision(), Recall(), AUC()])

In [37]:
# Define callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=10, mode='min')
lr_scheduler = LearningRateScheduler(lr_schedule)

seq_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 3)                 0         
                                                                 
 dense_7 (Dense)             (None, 32)                128       
                                                                 
 batch_normalization (BatchN  (None, 32)               128       
 ormalization)                                                   
                                                                 
 dense_8 (Dense)             (None, 16)                528       
                                                                 
 batch_normalization_1 (Batc  (None, 16)               64        
 hNormalization)                                                 
                                                                 
 dense_9 (Dense)             (None, 1)                

In [40]:
# Train the model
history = seq_model.fit(X_train_sc, y_train, epochs=500,
                        validation_data=(X_train_sc, y_train),
                        callbacks=[early_stop, lr_scheduler])

Epoch 1/500
9/9 [==============================] - 0s 25ms/step - loss: 0.5149 - accuracy: 0.7500 - precision: 0.6610 - recall: 0.4382 - auc: 0.7646 - val_loss: 0.5154 - val_accuracy: 0.7036 - val_precision: 0.5417 - val_recall: 0.4382 - val_auc: 0.7683 - lr: 0.0100
Epoch 2/500
9/9 [==============================] - 0s 26ms/step - loss: 0.5150 - accuracy: 0.7571 - precision: 0.6780 - recall: 0.4494 - auc: 0.7713 - val_loss: 0.5149 - val_accuracy: 0.7393 - val_precision: 0.7500 - val_recall: 0.2697 - val_auc: 0.7819 - lr: 0.0100
Epoch 3/500
9/9 [==============================] - 0s 18ms/step - loss: 0.5243 - accuracy: 0.7357 - precision: 0.6923 - recall: 0.3034 - auc: 0.7510 - val_loss: 0.5609 - val_accuracy: 0.7286 - val_precision: 0.8824 - val_recall: 0.1685 - val_auc: 0.7625 - lr: 0.0100
Epoch 4/500
9/9 [==============================] - 0s 19ms/step - loss: 0.4883 - accuracy: 0.7321 - precision: 0.6458 - recall: 0.3483 - auc: 0.8013 - val_loss: 0.5113 - val_accuracy: 0.7429 - val_pr

In [ ]:
# seq_model.fit(X_train_sc, y_train, epochs=1000)

In [42]:
# 결 , 결과치
score = seq_model.evaluate(X_test_sc, y_test)
score

4/4 [==============================] - 0s 6ms/step - loss: 0.8081 - accuracy: 0.6333 - precision: 0.4062 - recall: 0.3421 - auc: 0.5388


[0.8080738186836243,
 0.6333333253860474,
 0.40625,
 0.34210526943206787,
 0.5388318300247192]